<a href="https://colab.research.google.com/github/anikesh-17/Movie-Recommendation/blob/main/Movie_Recommendations.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [13]:
import pandas as pd
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity


file_path = 'imdb_top_1000.csv'
movies_df = pd.read_csv(file_path)


num_users = 1000
num_movies = movies_df.shape[0]


np.random.seed(42)
ratings_matrix = np.random.randint(1, 11, size=(num_users, num_movies)).astype(float)


mask = np.random.rand(num_users, num_movies) < 0.7
ratings_matrix[mask] = np.nan


ratings_df = pd.DataFrame(ratings_matrix, columns=movies_df['Series_Title'])


ratings_mean = ratings_df.mean(axis=1)
normalized_ratings = ratings_df.subtract(ratings_mean, axis='rows')


normalized_ratings_filled = normalized_ratings.fillna(0)


user_similarity = cosine_similarity(normalized_ratings_filled)
user_similarity_df = pd.DataFrame(user_similarity, index=ratings_df.index, columns=ratings_df.index)


def predict_ratings(ratings, similarity):
    mean_user_rating = ratings.mean(axis=1).to_numpy().reshape(-1, 1)
    ratings_diff = np.nan_to_num(ratings.to_numpy() - mean_user_rating)
    pred = mean_user_rating + similarity.dot(ratings_diff) / np.array([np.abs(similarity).sum(axis=1)]).T
    return pred

# Predict ratings
predicted_ratings = predict_ratings(normalized_ratings_filled, user_similarity)
predicted_ratings_df = pd.DataFrame(predicted_ratings, index=ratings_df.index, columns=ratings_df.columns)


def recommend_movies(user_index, num_recommendations=10):
    user_ratings = predicted_ratings_df.iloc[user_index].sort_values(ascending=False)
    recommended_movies = user_ratings.head(num_recommendations).index
    return recommended_movies


recommended_movies_user_0 = recommend_movies(0, 10)
print(recommended_movies_user_0)


Index(['Andaz Apna Apna', 'Terminator 2: Judgment Day',
       'La double vie de Véronique', 'Paan Singh Tomar', 'The Fighter',
       'Black Hawk Down', 'Nefes: Vatan Sagolsun', 'Dil Bechara', 'Malcolm X',
       'Sleepers'],
      dtype='object', name='Series_Title')


In [14]:
# Normalize the ratings (subtract mean rating for each movie)
ratings_mean_items = ratings_df.mean(axis=0)
normalized_ratings_items = ratings_df.subtract(ratings_mean_items, axis='columns')


normalized_ratings_items_filled = normalized_ratings_items.fillna(0)


item_similarity = cosine_similarity(normalized_ratings_items_filled.T)
item_similarity_df = pd.DataFrame(item_similarity, index=ratings_df.columns, columns=ratings_df.columns)


def predict_ratings_item(ratings, similarity):
    pred = ratings.dot(similarity) / np.array([np.abs(similarity).sum(axis=1)])
    return pred

# Predict ratings
predicted_ratings_items = predict_ratings_item(normalized_ratings_items_filled, item_similarity)
predicted_ratings_items_df = pd.DataFrame(predicted_ratings_items, index=ratings_df.index, columns=ratings_df.columns)

# Function to recommend top N movies for a given user based on item-based filtering
def recommend_movies_item(user_index, num_recommendations=10):
    user_ratings = predicted_ratings_items_df.iloc[user_index].sort_values(ascending=False)
    recommended_movies = user_ratings.head(num_recommendations).index
    return recommended_movies

# Get recommendations for the first user based on item-based filtering
recommended_movies_item_user_0 = recommend_movies_item(0, 10)
print(recommended_movies_item_user_0)


Index(['The Shawshank Redemption', 'The Godfather', 'The Dark Knight',
       'The Godfather: Part II', '12 Angry Men',
       'The Lord of the Rings: The Return of the King', 'Pulp Fiction',
       'Schindler's List', 'Inception', 'Fight Club'],
      dtype='object', name='Series_Title')
